# Scraping med Selenium i python

`Selenium` er et enkelt scraping-bibliotek i python. Det bruker en webdriver som håndterer oppkobling, navigasjon og ekstaksjon av data fra netsider. 
[Dokumentasjon](https://selenium-python.readthedocs.io/index.html/)

Passer godt for:
* Enkle til mer avanserte oppgaver
* Situasjoner hvor man må interagere med nettsiden som scrapes

Fordeler:
* Svært Enkel i bruk 
* Automatisk gjennkjenning av HTML/CSS kode
* Håndterer javascript og interaksjon med de ulike elementene på siden 

Ulemper:
* For interaksjon med større systemer/svært gjentakende oppgaver kan det være anbefalt å bruke `scrapy` med en extension for javascript for å løse oppgavene 
* Litt treg ettersom den laster inn hele sider heller enn å bare trekke ut HTML

Her brukes selenium i sin standardversjon, men man kan også bruke webdriveren fra `selenium-respectful` for å håndtere begrensninger satt i robots.txt på ulike nettsider ([Lenke](https://pypi.org/project/selenium-respectful/)). NB. dette krever Redis og vil ikke bli gjennomgått i denne sesjonen. For små enkle prosjekter kan pålagt ventetid/request per sekund håndteres med `time`-modulen 

## Webdriver
For å bruke selenium må du laste ned en webdriver som passer til din nettleser. Bruker du PwC PC anbefales bruk av Firefox ettersom chrome driveren får problemer med brannmuren. 
1. Last ned firefox på din pc (hvis du ikke har dette fra før)
2. Last ned den tilhørende wedriveren geckodriver [herfra](https://github.com/mozilla/geckodriver/releases)

## Enkelt kodeeksempel

Oppgave: Søk på "Nakstad" på nrk.no og hent ut alle søkeresultatene med tilhørende lenker

In [1]:
from selenium import webdriver # Webdriver-modul som lar deg interagere med nettsider
from selenium.webdriver.common.keys import Keys # Lar deg burke taster, feks. enter-knappen
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [8]:
# Definere filsti til din webdriver lokalt på pcen, her ligger den for enkelhet i samme directory 
PATH = "geckodriver.exe"

# Sette opp driveren som skal brukes
driver = webdriver.Firefox(executable_path = PATH)

# Hent siden vi er interessert i 
driver.get("https://www.nrk.no/")

# Lokaliser søkefelt. Inspeksjon av HTML viser at denne har "name = q"
search = driver.find_element_by_name("q")

# Skriv inn ønsket tekst i søkefelt 
search.send_keys("nakstad")

# Trykk enter
search.send_keys(Keys.RETURN)

Vi er interessert i resultatene fra nrk.no, men ikke alle de nyeste søkeresultatene vises på siden. Vi vil derfor trykke på "+ vis mer fra NRK.no"-knappen for å få opp alle de nyeste treffene. Det kan ta varierende tid å laste inn siden med søkeresultatene. Det kan derfor være greit å legge inn en liten buffer slik at vi er sikker på at elementene er lastet på siden før vi gjør nye spørringer. 

In [9]:
# Superenkel måte gjøre det på (1)
# Vent fem sekunder, bør være nok til at siden er lastet
time.sleep(5)

# Finn knapp-elementer 
buttons = driver.find_elements_by_class_name("nrkno-search-button") 

# Trykk på den fjerne for å vise flere treff fra NRK.no
buttons[3].click()

In [6]:
# Superenkel måte gjøre det på (1)
#try:
#    element = WebDriverWait(driver, 20).until( # Vent maks 20 sekunder 
#        EC.presence_of_element_located((By.TAG_NAME, "main")) # Inntil driveren finner element main
#    )
#    buttons = driver.find_elements_by_class_name("nrkno-search-button") # Finn button-elementer
#    buttons[3].click() # Trykk på den ferde (som er den vi vil ha)
#except:
#    driver.quit() # Avslutt driveren dersom det tar for lang tid

In [10]:
# Finner alle grupper av søketreff (grupperte søkeresultater fra radio, nett, p3 etc.)
sections = driver.find_elements_by_class_name("nrkno-search-group")

# Nyheter fra NRK.no er den fjerde gruppen, henter deretter ut hver artikkel 
articles = sections[3].find_elements_by_class_name("nrkno-search-hit")

# Holder informasjonen som skal returneres
results = pd.DataFrame(columns = ['Overskrift','Ingress','Lenke'])

# looper gjennom treffene på NRK.no
for article in articles:
    
    # Overskrift
    title = article.find_element_by_class_name("nrkno-search-hit__link").text 
    
    # Lenke 
    link = article.find_element_by_class_name("nrkno-search-hit__link").get_attribute("href")
    
    # Ingress
    excp = article.find_element_by_class_name("nrkno-search-hit__excerpt").text 
    
    # Lagrer resultatene
    results = results.append(pd.Series(
        [title, excp, link], index = ['Overskrift','Ingress','Lenke']), ignore_index = True) 
    

# Avslutter driveren
driver.quit() 

# Vis søketreffene 
results

,Overskrift,Ingress,Lenke
0,Regjeringen opphever tiltaksnivåene på Østlandet,Regjeringen opphever tiltaksnivåene for østlan...,https://www.nrk.no/norge/regjeringen-opphever-...
1,Nakstad: Så lave må smittetallene være for at ...,Assisterende helsedirektør Espen Rostrup Nakst...,https://www.nrk.no/norge/nakstad_-ma-godt-unde...
2,Nakstad: – Tror ikke folk i Oslo forstår alvoret,Smittetallene i Oslo går jevnt oppover. Nå ber...,https://www.nrk.no/norge/helsemyndighetene-er-...
3,FHI estimerer R-tallet til 1: – Smittekurven h...,For første gang på fem uker er det påvist færr...,https://www.nrk.no/norge/faerre-nye-koronatilf...
4,Nakstad: Kan komme nye tiltak for barn og unge,Neste uke skal rådene og retningslinjene i sko...,https://www.nrk.no/norge/nakstad_-kan-komme-ny...
5,Legar hyller Nakstad med eksklusive plagg,To legar vil få folk til å trykke Nakstad til ...,https://www.nrk.no/osloogviken/overlege-sel-na...
6,Nakstad: – De neste to månedene kommer til å b...,Mars og april kan bli de vanskeligste i hele p...,https://www.nrk.no/norge/nakstad_-_-de-neste-t...
7,Nakstad: Mye tyder på at andre smittebølge er ...,Mandag ble det registrert 282 nye koronasmitte...,https://www.nrk.no/norge/nakstad_-mye-tyder-pa...
8,Nakstad frykter ny smittebølge når vaksinen ko...,Vi kan få en ny smittebølge dersom vi slapper ...,https://www.nrk.no/norge/frykter-ny-smittebolg...
9,Nakstad: – Gir et håp om færre sykehusinnlegge...,En uke med koronatesting etter påske viser ing...,https://www.nrk.no/norge/nakstad_-_-gir-et-hap...
